In [0]:
#Mount google drive to ream ngram statistics
from google.colab import drive
drive.mount('/content/drive')


In [83]:
ngram_location= "/content/drive/My Drive/Colab Notebooks/Mystery Twister/ngrams/"
!ls "./drive/My Drive/Colab Notebooks/Mystery Twister/ngrams/"


english_bigrams.txt    english_quadgrams.txt
english_monograms.txt  english_trigrams.txt


In [2]:
# read strips from url

import requests

url = 'https://www.mysterytwisterc3.org/images/challenges/mtc3-schmeh-09-stripset.txt'

r = requests.post(url)

#print (r.text)
STRIPS = []

for line in r.text.splitlines():
    STRIPS.append(line[3:])

print (STRIPS)

['DTYLUKCNOIVSEPFQJWRGMBZAXH', 'LOGKWPRMDUVSYFJNTQIHEAZCBX', 'YGSKCJETRMUWNHQXIZFAOVBLDP', 'BERHJYKWCSLNPDZIGFUAOXVQTM', 'JLSGAOPZEMBVQCUIYDTHXRWFKN', 'IPWSURJTOQGEVDBMYKLNHAXZFC', 'CGLXSIBTJHOKNDMQPVRZAYEFUW', 'YGHKDVLQEXUOASZWPJFBCNRITM', 'ACZLSOGEDPYWFXHBVIMUKNTJQR', 'XTHNIBAFEUQSGLJDWOZKMPVCYR', 'MFKGNURPJZTBQWLCASIHVYOXDE', 'NHTEPCFDXRYZBAIMSGVJKUOQWL', 'AHFPGVUKLMNCTSRDEIWZXYQBOJ', 'SAHIKWDQJNPVUTZCBYLOGFMREX', 'GPNWMTOSQHJVYKFEALXCIRDBZU', 'GPCWQOSVZKINJHERUABDTMXLFY', 'NKAELXYVRDOGZIMTFUBSPJQHWC', 'EFTXLBCAWHUJGVOMYRSNKDQIZP', 'ZLNRQBAVYMJUDOTSHXCWGPKIEF', 'YAGDUTFIXMBWLOJQVNERHKSZPC', 'THRAWJEMNFKYZCIGOBXVSULDQP', 'INESDOMTPBQGHYFUZCRVWLKJXA', 'YIRAELHQOSXGCWTPJVZUNFMDKB', 'ITLFAXYCMOPVGZHURDBWNSJQKE', 'QRJGEFWVLKSUHPCXYBMOITNZAD', 'VYGNODPUJEMFCZIXASKBWHRTLQ', 'ANZIFPCLOKMHSJDEWVBYQXTURG', 'NEBZCRMKDPATGLSOWHFIJQVUYX', 'EQJTZFYSINAUBWCVDXRKHPLGMO', 'KDOTRBZWIAUYPLNHEGFQSJMCXV', 'XVSRDIUZFCQTMPLHYEGJAKNWOB', 'MVELIBQAFGJNPCKWXUSTZRDYHO', 'BMOITNEWDAUVPHZJXYSLRQCFGK', 'UYWVFPNH

In [0]:
# testdata from M-138 challenge 2
# BBQGFHSDXNFKLXRREYYPADREW TFRJGJDCBGDZFXINXMWYLHTGP AXHOLTHXPRCTTADFWOJYXAEYR NKRXRXDKHSFDUVPXQGWMKMYKZ
# TWOTHINGSAREINFINITETHEUN IVERSEANDHUMANSTUPIDITYCO NGRATULATIONYOUHAVEDECIPH EREDANALBERTEINSTEINQUOTE

CHAL2CIPHER="BBQGFHSDXNFKLXRREYYPADREWTFRJGJDCBGDZFXINXMWYLHTGPAXHOLTHXPRCTTADFWOJYXAEYRNKRXRXDKHSFDUVPXQGWMKMYKZ"
CHAL2PLAIN ="TWOTHINGSAREINFINITETHEUNIVERSEANDHUMANSTUPIDITYCONGRATULATIONYOUHAVEDECIPHEREDANALBERTEINSTEINQUOTE"

# challenge3 ciphertext
CHAL3CIPHER="RIGVRXIXRHZQOGDQYIXVHCZKJLCDKUSGNDPPIBCLGPZBRUTRFJXHTNQPHWXGQAXPKEEEKMDPWFKSDTLKPTFIXIRUXNTIMTZQQCQOSOPFBXFMMZPSIGZSANJKYHWIO"

In [0]:
#decrypt function

def decrypt(ciphertext,stripnumber,offset):
  l=0
  decrypted_text=""
  for c in ciphertext:
    
    currentstrip=STRIPS[stripnumber]
    decrypted_text+=currentstrip[(currentstrip.index(c)-offset)%26]
    l+=1
  return (decrypted_text) 



def decrypt_text(cipher,key,offset):
  l=0
  decrypted_text=""
  for c in cipher:
    
    currentstrip=STRIPS[key[l%25]]
    decrypted_text+=currentstrip[(currentstrip.index(c)-offset)%26]
    l+=1
  return (decrypted_text)

In [0]:
#ngram score functions
#to rate decrypted text (http://practicalcryptography.com/cryptanalysis/text-characterisation/quadgrams/)

'''
Allows scoring of text using n-gram probabilities
17/07/12
'''
from math import log10

class ngram_score(object):
    def __init__(self,ngramfile,sep=' '):
        ''' load a file containing ngrams and counts, calculate log probabilities '''
        self.ngrams = {}
        f = open(ngramfile, "r")
        lines = list(f)
        f.close()
        for line in lines:
            key,count = line.split(sep) 
            self.ngrams[key] = int(count)
        self.L = len(key)
        self.N = sum(self.ngrams.values())
        #calculate log probabilities
        for key in self.ngrams.keys():
            self.ngrams[key] = log10(float(self.ngrams[key])/self.N)
        self.floor = log10(0.01/self.N)
        #print (self.ngrams)
        
    def score(self,text):
        ''' compute the score of text '''
        score = 0
        ngrams = self.ngrams.__getitem__
        for i in range(len(text)-self.L+1):
            if text[i:i+self.L] in self.ngrams: score += ngrams(text[i:i+self.L])
            else: score += self.floor          
        return score

In [107]:


fitness_bi = ngram_score(ngram_location+"english_monograms.txt")     # load bigram statistics

      
#print fitnesscores testvectors
print(fitness_bi.score(CHAL2CIPHER))
print(fitness_bi.score(CHAL2PLAIN))

print(fitness_bi.score(CHAL3CIPHER))

-168.4008782469253
-124.73306104261682
-215.6085721938005


In [149]:
fitness_quad = ngram_score(ngram_location+"english_quadgrams.txt")     # load quadgram statistics

      
#print fitnesscores testvectors
print(fitness_quad.score(CHAL2CIPHER))
print(fitness_quad.score(CHAL2PLAIN))

print(fitness_quad.score(CHAL3CIPHER))
print(fitness_quad.score("TWOTIVERNGRAERED"))


-824.2064992172529
-408.2884205310118
-1072.2580170736428
-62.52787906763904


In [131]:
# try to find possible offset, use monogram scores on ecrypted chars:
# chars 0,25,50,75,100 use the same strip
# decrypt these chars and score the monograms

cipher=CHAL2CIPHER

L=[]
for n in range(25):
  
  selected_chars=cipher[n]+cipher[n+25]+cipher[n+50]+cipher[n+75] #+cipher[n+100]
  #print (selected_chars)
  for s in range(100):
    for offset in range(25):
      decrypted_text=decrypt(selected_chars,s,offset)
      r = fitness_bi.score(decrypted_text)
      if  r > -4.5:
        #print("strip: " + str(s) + " offset: " + str(offset) + " decypted :" + decrypted_text + " score: " +str(r))
        L.append(offset)

        
for o in range(25):
  print ("offset " + str(o) + " occured : " + str(L.count(o)))

offset 0 occured : 0
offset 1 occured : 19
offset 2 occured : 18
offset 3 occured : 13
offset 4 occured : 24
offset 5 occured : 26
offset 6 occured : 31
offset 7 occured : 26
offset 8 occured : 18
offset 9 occured : 20
offset 10 occured : 18
offset 11 occured : 26
offset 12 occured : 24
offset 13 occured : 20
offset 14 occured : 17
offset 15 occured : 21
offset 16 occured : 20
offset 17 occured : 24
offset 18 occured : 17
offset 19 occured : 21
offset 20 occured : 24
offset 21 occured : 29
offset 22 occured : 16
offset 23 occured : 11
offset 24 occured : 22


In [135]:
# decrypt test CHAL2 challenge

CIPHER=CHAL2CIPHER
KEY= [95, 5, 30, 99, 69, 49, 70, 62, 48, 81, 53, 60, 8, 25, 72, 19, 68, 88, 22, 75, 4, 29, 54, 16, 90]
OFFSET=12

print (decrypt_text(CIPHER,KEY,OFFSET))


TWOTHINGSAREINFINITETHEUNIVERSEANDHUMANSTUPIDITYCONGRATULATIONYOUHAVEDECIPHEREDANALBERTEINSTEINQUOTE


In [0]:
import random
#first attempt CHAL2 hill climbing
# initialize quads
quads = ngram_score(ngram_location+"english_quadgrams.txt")     # load quadgram statistics

#initialize max_score with the cipher
max_score= quads.score(CIPHER)
#init random KEY
KEY=[]
while len(KEY)<100:
  r = random.randint(0, 99)
  if r not in KEY:
    KEY.append(r)
print (KEY)

for i in range (10000):
  plain=decrypt_text(CIPHER,KEY,OFFSET)
  t=quads.score(plain)
  if t>max_score:
    max_score=t
    print ("new max: " + str(max_score) + " : "+plain)
  else:
    # swap keys
    k=random.randint(0, 99)
    while k in KEY:
      k=random.randint(0, 99)
    n=random.randint(0, 25)
    KEY[n]=k
    
   
    
    
  



In [155]:
#brute force
CIPHER=CHAL2CIPHER
OFFSET=12
import random

#first attempt CHAL2 
# initialize quads
quads = ngram_score(ngram_location+"english_quadgrams.txt")     # load quadgram statistics

#initialize max_score with the cipher
max_score= quads.score(CIPHER)
#init random KEY
for i in range(100):
  KEY.append(i)

t=0
max_score=-999990

for i in range (1000000):
  
  random.shuffle(KEY)
  plain=decrypt_text(CIPHER,KEY,OFFSET)
  t=quads.score(plain)
  if t>max_score:
    max_score=t
    print ("new max: " + str(max_score) + " : "+plain)
  
    
  

new max: -812.3691703411904 : JVFJBCYZPTLZIULSQEVMXQLGTLHEUOEOVQVFSAUQJTCZNTPACKYDBCJIZDKIEJYNFVAISNPXWWLUFESIPOITBLXEDIYGAZKBRTXI
new max: -777.7253766623483 : LWYXAMBUSIKDRQOAJVWOWPEFUOCLPKINOPYLNDQYMMRCGTOBWXZBOIISXWLPGWUWUDLLTGSVIKMDHLLXTNZXLKSFZKWSACVORDRS
new max: -742.9990227465149 : ULDEUJTJWITFUBWUKGNHYPMTVFNIFOVVSYSLJMBHXNUCSZQUCMOAXDZEPOVDSMANNLDPPSAHISXSJINFIVPKETRHJTIMVCFFWAJT
new max: -742.8783726308295 : RYITHRRPBSEQIROPGGSLUFQTIEZCBNFNAWANTRRRIDXDUPDSAUUUKRQIWIJMHIQSYXMIEUOHGXLIMCDGHNQLDEYGQCSOFDHEBWPA
new max: -738.548999008547 : EMDEHECZMLWPYLPGTGPEQUIKQJWIPITHJKAKDHLEOCXZWYGXPUVNATONIFHONORFADZIDWSBOSECZIRSUHSZNWNEHJIHFZPMSGXI
new max: -737.1645658193316 : YWIRHPXKTIGZMBMUTNOREYQALEVFHSHNNIYSWYBFMOJUERGWOWFGAMTAGMDPZIPNEBSQFEUBOIMMRFAAINAPLGUQJWPCIUWLULFS
new max: -710.2931969147531 : KPWAMOBNIISESYCTIGFOYLMBMALDRRQTFSSRIAYPHSLIGBKSFRCWLKYEFKRVFVIDGIXVLGHSOMGRADNKMTZPUSRLWHENSIVQUUHK
new max: -703.409804765247 : RYPONETSWALMMMWOFNESRRUWJELITUTVBGRAXYMCHDWNDUWASKHSRQS